# David Bobek

In [2]:
import numpy as np


def read_sequences(file_path):
    """
    Read sequences from a text file.

    Args:
    file_path (str): Path to the text file containing sequences.

    Returns:
    list: List of sequences.
    """
    sequences = []
    with open(file_path, 'r') as file:
        for line in file:
            sequences.append(line.strip())
    return sequences

def calculate_position_count_matrix(sequences):
    """
    Calculate position-specific count matrix.

    Args:
    sequences (list): List of sequences.

    Returns:
    numpy.ndarray: Position-specific count matrix.
    """
    num_sequences = len(sequences)
    sequence_length = len(sequences[0])
    position_count_matrix = np.zeros((4, sequence_length)) # Assuming DNA sequences
    
    for i in range(sequence_length):
        position_counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        for sequence in sequences:
            nucleotide = sequence[i]
            position_counts[nucleotide] += 1
        for j, nucleotide in enumerate('ACGT'):
            position_count_matrix[j, i] = position_counts[nucleotide]
    
    return position_count_matrix

def calculate_position_frequency_matrix(position_count_matrix):
    """
    Calculate position-specific frequency matrix.

    Args:
    position_count_matrix (numpy.ndarray): Position-specific count matrix.

    Returns:
    numpy.ndarray: Position-specific frequency matrix.
    """
    sequence_length = position_count_matrix.shape[1]
    position_frequency_matrix = np.zeros_like(position_count_matrix)
    
    for i in range(sequence_length):
        total_counts = np.sum(position_count_matrix[:, i])
        position_frequency_matrix[:, i] = position_count_matrix[:, i] / total_counts
    
    return position_frequency_matrix

def calculate_position_weight_matrix(position_frequency_matrix):
    """
    Calculate position-specific weight matrix.

    Args:
    position_frequency_matrix (numpy.ndarray): Position-specific frequency matrix.

    Returns:
    numpy.ndarray: Position-specific weight matrix.
    """
    sequence_length = position_frequency_matrix.shape[1]
    position_weight_matrix = np.zeros_like(position_frequency_matrix)
    
    for i in range(sequence_length):
        background_frequencies = {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25} # Assuming equal background frequencies
        for j, nucleotide in enumerate('ACGT'):
            position_weight_matrix[j, i] = np.log2(position_frequency_matrix[j, i] / background_frequencies[nucleotide])
    
    return position_weight_matrix

sequences = read_sequences("ATBI_pssm 3.txt")

position_count_matrix = calculate_position_count_matrix(sequences)
print("Position-specific count matrix:")
print(position_count_matrix)

position_frequency_matrix = calculate_position_frequency_matrix(position_count_matrix)
print("\nPosition-specific frequency matrix:")
print(position_frequency_matrix)

position_weight_matrix = calculate_position_weight_matrix(position_frequency_matrix)
print("\nPosition-specific weight matrix:")
print(position_weight_matrix)


Position-specific count matrix:
[[29. 19. 25. 33. 11. 29.]
 [15. 23. 21. 21. 32. 20.]
 [20. 31. 30. 27. 33. 28.]
 [36. 27. 24. 19. 24. 23.]]

Position-specific frequency matrix:
[[0.29 0.19 0.25 0.33 0.11 0.29]
 [0.15 0.23 0.21 0.21 0.32 0.2 ]
 [0.2  0.31 0.3  0.27 0.33 0.28]
 [0.36 0.27 0.24 0.19 0.24 0.23]]

Position-specific weight matrix:
[[ 0.21412481 -0.39592868  0.          0.40053793 -1.18442457  0.21412481]
 [-0.73696559 -0.12029423 -0.25153877 -0.25153877  0.35614381 -0.32192809]
 [-0.32192809  0.31034012  0.26303441  0.11103131  0.40053793  0.16349873]
 [ 0.52606881  0.11103131 -0.05889369 -0.39592868 -0.05889369 -0.12029423]]


In [3]:
%pip install blosum
import blosum as bl

Note: you may need to restart the kernel to use updated packages.


In [4]:


class Alignment:
    def __init__(self, seq1, seq2, substitution_matrix, gap_penalty):
        self.seq1 = seq1
        self.seq2 = seq2
        self.substitution_matrix = substitution_matrix
        self.gap_penalty = gap_penalty
        self.alignment_matrix = [[0] * (len(seq2) + 1) for _ in range(len(seq1) + 1)]

    def needleman_wunsch(self):
        # Initialize the first row and first column of the alignment matrix
        for i in range(1, len(self.seq1) + 1):
            self.alignment_matrix[i][0] = self.alignment_matrix[i - 1][0] + self.gap_penalty
        for j in range(1, len(self.seq2) + 1):
            self.alignment_matrix[0][j] = self.alignment_matrix[0][j - 1] + self.gap_penalty

        # Fill the rest of the alignment matrix
        for i in range(1, len(self.seq1) + 1):
            for j in range(1, len(self.seq2) + 1):
                match_score = self.substitution_matrix[self.seq1[i - 1]][self.seq2[j - 1]]
                match = self.alignment_matrix[i - 1][j - 1] + match_score
                delete = self.alignment_matrix[i - 1][j] + self.gap_penalty
                insert = self.alignment_matrix[i][j - 1] + self.gap_penalty
                self.alignment_matrix[i][j] = max(match, delete, insert)

    def smith_waterman(self):
        # Initialize the first row and first column of the alignment matrix
        # In Smith-Waterman, the first row and column are initialized to 0, which is already done by default

        # Fill the rest of the alignment matrix
        for i in range(1, len(self.seq1) + 1):
            for j in range(1, len(self.seq2) + 1):
                # Calculate match score using the substitution matrix
                match_score = self.substitution_matrix[self.seq1[i - 1]][self.seq2[j - 1]]
                # Possible scores for the current cell
                match = self.alignment_matrix[i - 1][j - 1] + match_score
                delete = self.alignment_matrix[i - 1][j] + self.gap_penalty
                insert = self.alignment_matrix[i][j - 1] + self.gap_penalty
                # Select the maximum score, reset to zero if all scores are negative
                self.alignment_matrix[i][j] = max(0, match, delete, insert)

                

    def print_alignment_matrix(self):
        print("   " + " ".join(["{:>4s}".format("-")] + [x for x in self.seq2]))
        for i in range(len(self.alignment_matrix)):
            if i == 0:
                print("{:2s}".format("-"), end=" ")
            else:
                print("{:2s}".format(self.seq1[i - 1]), end=" ")
            for j in range(len(self.alignment_matrix[i])):
                print("{:>4.1f}".format(self.alignment_matrix[i][j]), end=" ")
            print()


# Example usage
seq1 = "HEAGAWGHEE"
seq2 = "PAWHEAE"
substitution_matrix = bl.BLOSUM(50)
gap_penalty = -8

nw_alignment = Alignment(seq1, seq2, substitution_matrix, gap_penalty)
nw_alignment.needleman_wunsch()
print("Needleman-Wunsch Alignment Matrix:")
nw_alignment.print_alignment_matrix()

sw_alignment = Alignment(seq1, seq2, substitution_matrix, gap_penalty)
sw_alignment.smith_waterman()
print("\nSmith-Waterman Alignment Matrix:")
sw_alignment.print_alignment_matrix()


print("Needleman-Wunsch Alignment:")
def needleman_allign(seq1, seq2, substitution_matrix, gap_penalty, nw_alignment):
    i = len(seq1)
    j = len(seq2)
    aligned_seq1 = ""
    aligned_seq2 = ""

    while i > 0 or j > 0:
        if i > 0 and j > 0 and nw_alignment.alignment_matrix[i][j] == nw_alignment.alignment_matrix[i - 1][j - 1] + substitution_matrix[seq1[i - 1]][seq2[j - 1]]:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif i > 0 and nw_alignment.alignment_matrix[i][j] == nw_alignment.alignment_matrix[i - 1][j] + gap_penalty:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1
        
    print(aligned_seq1)
    print(aligned_seq2)

needleman_allign(seq1, seq2, substitution_matrix, gap_penalty, nw_alignment)

print("\nSmith-Waterman Alignment:")
def smith_allign(seq1, seq2, substitution_matrix, gap_penalty, sw_alignment):
    i = len(seq1)
    j = len(seq2)
    aligned_seq1 = ""
    aligned_seq2 = ""

    max_score = 0
    max_i = 0
    max_j = 0

    for i in range(len(seq1) + 1):
        for j in range(len(seq2) + 1):
            if sw_alignment.alignment_matrix[i][j] > max_score:
                max_score = sw_alignment.alignment_matrix[i][j]
                max_i = i
                max_j = j
            
    i = max_i
    j = max_j

    while i > 0 or j > 0:
        if i > 0 and j > 0 and sw_alignment.alignment_matrix[i][j] == sw_alignment.alignment_matrix[i - 1][j - 1] + substitution_matrix[seq1[i - 1]][seq2[j - 1]]:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif i > 0 and sw_alignment.alignment_matrix[i][j] == sw_alignment.alignment_matrix[i - 1][j] + gap_penalty:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        elif j > 0:
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1
        else:
            break  # This else can be useful if both i and j are zero, preventing infinite loops.

    
    
    print(aligned_seq1)
    print(aligned_seq2)

smith_allign(seq1, seq2, substitution_matrix, gap_penalty, sw_alignment)


        

Needleman-Wunsch Alignment Matrix:
      - P A W H E A E
-   0.0 -8.0 -16.0 -24.0 -32.0 -40.0 -48.0 -56.0 
H  -8.0 -2.0 -10.0 -18.0 -14.0 -22.0 -30.0 -38.0 
E  -16.0 -9.0 -3.0 -11.0 -18.0 -8.0 -16.0 -24.0 
A  -24.0 -17.0 -4.0 -6.0 -13.0 -16.0 -3.0 -11.0 
G  -32.0 -25.0 -12.0 -7.0 -8.0 -16.0 -11.0 -6.0 
A  -40.0 -33.0 -20.0 -15.0 -9.0 -9.0 -11.0 -12.0 
W  -48.0 -41.0 -28.0 -5.0 -13.0 -12.0 -12.0 -14.0 
G  -56.0 -49.0 -36.0 -13.0 -7.0 -15.0 -12.0 -15.0 
H  -64.0 -57.0 -44.0 -21.0 -3.0 -7.0 -15.0 -12.0 
E  -72.0 -65.0 -52.0 -29.0 -11.0  3.0 -5.0 -9.0 
E  -80.0 -73.0 -60.0 -37.0 -19.0 -5.0  2.0  1.0 

Smith-Waterman Alignment Matrix:
      - P A W H E A E
-   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 
H   0.0  0.0  0.0  0.0 10.0  2.0  0.0  0.0 
E   0.0  0.0  0.0  0.0  2.0 16.0  8.0  6.0 
A   0.0  0.0  5.0  0.0  0.0  8.0 21.0 13.0 
G   0.0  0.0  0.0  2.0  0.0  0.0 13.0 18.0 
A   0.0  0.0  5.0  0.0  0.0  0.0  5.0 12.0 
W   0.0  0.0  0.0 20.0 12.0  4.0  0.0  4.0 
G   0.0  0.0  0.0 12.0 18.0 10.0

In [5]:
from Bio import SeqIO
from Bio.Align import substitution_matrices

# Calculate score based on substitution matrix
def score_sequences(sequence_a, sequence_b, subs_matrix):
    if len(sequence_a) != len(sequence_b):
        raise ValueError("Both sequences should have equal length.")
    return sum(subs_matrix[sequence_a[pos], sequence_b[pos]] for pos in range(len(sequence_a)))

# Generate sequence variants based on amino acid substitutions
def generate_variants(base_sequence, possible_sequences, residue_set, subs_matrix, min_score):
    variant_dict = {}
    for seq in possible_sequences:
        index_in_base = base_sequence.index(seq)
        for position, residue in enumerate(seq):
            for mutation in residue_set:
                if mutation != residue:
                    new_variant = seq[:position] + mutation + seq[position+1:]
                    if score_sequences(seq, new_variant, subs_matrix) >= min_score:
                        variant_dict[new_variant] = index_in_base
    return variant_dict

# Scan the database for matches to query
def scan_database(database, query, positions_dict, span, score_threshold, subs_matrix):
    match_info = {}
    for start in range(len(database) - span + 1):
        db_segment = database[start:start + span]
        if db_segment in positions_dict:
            sequence_start = max(0, start - positions_dict[db_segment])
            sequence_end = min(len(database), sequence_start + len(query))
            full_segment = database[sequence_start:sequence_end]
            if len(full_segment) == len(query):
                match_score = score_sequences(full_segment, query, subs_matrix)
                if match_score > score_threshold:
                    match_info[full_segment] = (sequence_start, match_score)
    return match_info

# Main function to run the BLAST-like search
def run_blast_search(db_seq, query_seq, window_size, min_similarity_score):
    substitution_matrix = substitution_matrices.load("BLOSUM62")
    amino_acid_list = "ACDEFGHIKLMNPQRSTVWY"
    query_positions = {query_seq[i:i + window_size]: i for i in range(len(query_seq) - window_size + 1)}
    potential_variants = generate_variants(query_seq, list(query_positions.keys()), amino_acid_list, substitution_matrix, min_similarity_score)
    all_variants = query_positions.copy()
    all_variants.update(potential_variants)
    matching_results = scan_database(db_seq, query_seq, all_variants, window_size, min_similarity_score, substitution_matrix)

    output = []
    for sequence, (position, score) in matching_results.items():
        output.append({
            "matched_sequence": sequence,
            "position": position,
            "alignment_score": score
        })
    return output

# Read FASTA file to get sequences
def load_sequences_from_fasta(filepath):
    sequence_data = {}
    for seq_record in SeqIO.parse(filepath, "fasta"):
        sequence_data[seq_record.id] = str(seq_record.seq)
    return sequence_data

# Example usage

fasta = load_sequences_from_fasta("ATBI_BLAST_exemplary input db.fasta")
database_seq = list(fasta.values())[0]
target_seq = "SIRFVRLI"
window = 3
threshold = 5

res= run_blast_search(database_seq, target_seq, window, threshold)

print("Matching results:")
for result in res:
    print(result)
    

Matching results:
{'matched_sequence': 'RIRSVYPV', 'position': 298, 'alignment_score': 8.0}
{'matched_sequence': 'TIAFGGCV', 'position': 402, 'alignment_score': 7.0}
{'matched_sequence': 'TSAFVETV', 'position': 483, 'alignment_score': 10.0}
{'matched_sequence': 'AARVVRSI', 'position': 539, 'alignment_score': 15.0}
{'matched_sequence': 'AQNSVRVL', 'position': 554, 'alignment_score': 8.0}
{'matched_sequence': 'SLRLIDAM', 'position': 576, 'alignment_score': 12.0}
{'matched_sequence': 'SQYSLRLI', 'position': 573, 'alignment_score': 11.0}
{'matched_sequence': 'GVEFLRDG', 'position': 637, 'alignment_score': 7.0}
{'matched_sequence': 'DITFLKKD', 'position': 1274, 'alignment_score': 7.0}
{'matched_sequence': 'GIEFLKRG', 'position': 1523, 'alignment_score': 7.0}
{'matched_sequence': 'SLREVRTI', 'position': 1560, 'alignment_score': 20.0}
{'matched_sequence': 'AANFCALI', 'position': 1706, 'alignment_score': 12.0}
{'matched_sequence': 'NIKFADDL', 'position': 1933, 'alignment_score': 9.0}
{'matched